Recommended Top 10 Open Source Art Museums

# Art Museum Country Data License Data Access Recommended Use

1️⃣ The Metropolitan Museum of Art (The Met) 🇺🇸 USA CC0 API

/ CSV Largest, free, high-quality images

2️⃣ Rijksmuseum 🇳🇱 Netherlands CC0 API

Excellent portraits with light and shadow and distinctive clothing features

3️⃣ Tate Britain 🇬🇧 UK CC-BY-NC GitHub CSV

Pre-Raphaelite, Rossetti (key focus)

4️⃣ National Gallery (London) 🇬🇧 UK CC BY API
Core of European classical portraiture

5️⃣ Art Institute of Chicago (AIC) 🇺🇸 USA CC0 API

High-quality portraits, American Impressionism

6️⃣ Cleveland Museum of Art (CMA) 🇺🇸 USA CC0 API
Baroque to Modern
7️⃣ National Gallery of Art (Washington, NGA) 🇺🇸 USA CC0 API
Complete 18th–19th century portrait data
8️⃣ Victoria and Albert Museum (V&A) 🇬🇧 UK Open API
Extensive images of fashion and body posture
9️⃣ Museu Nacional d’Art de Catalunya (MNAC) 🇪🇸 Spain CC BY API
Gothic/Baroque figure data
0️⃣ National Gallery of Australia (NGA-AUS) 🇦🇺 Australia CC BY API / CSV
Expanding non-Eurocentric perspectives

In [ ]:
# main.py
# Embodied Aesthetic Reconstruction — Streamlit App (Resonance-first UI)
# --------------------------------------------------------------------
# - CLIP + Pose + Color + Significance fusion
# - Human-centered UI: semantic "resonance" labels instead of raw numeric scores
# - Sidebar toggle "Debug mode" to show raw similarity when needed
# - Significance score (0–100) + badges + "Learn more"
# - Extra metadata fields: price_estimate_usd, significance_text, interpretive_note_cn
# - iCloud placeholder avoidance, robust pathing, cached embeddings
# - YOLOv8n-pose via HuggingFace (optional), proper device mapping for MPS/CUDA/CPU
# --------------------------------------------------------------------

from __future__ import annotations
import io
import json
import math
import time
from pathlib import Path
from typing import List, Tuple, Optional

import numpy as np
import pandas as pd
from PIL import Image, ImageOps
import streamlit as st

# -------------------- Torch / device --------------------
try:
    import torch
    TORCH_OK = True
except Exception:
    TORCH_OK = False

def get_device() -> str:
    """Return 'mps' on Apple Silicon, 'cuda' on Nvidia, else 'cpu'."""
    if not TORCH_OK:
        return "cpu"
    try:
        if torch.backends.mps.is_available():
            return "mps"
        if torch.cuda.is_available():
            return "cuda"
    except Exception:
        pass
    return "cpu"

DEVICE = get_device()
if TORCH_OK:
    try:
        torch.set_float32_matmul_precision("medium")
    except Exception:
        pass

def yolo_device() -> str:
    """Map torch device to Ultralytics' expected device string."""
    if DEVICE == "mps":
        return "mps"
    if DEVICE == "cuda":
        return "0"
    return "cpu"

# -------------------- Paths --------------------
APP_DIR = Path(__file__).parent.resolve()
DATA_DIR = APP_DIR / "data"
IMAGES_DIR = DATA_DIR / "images"
INTERIM_DIR = DATA_DIR / "interim"
CACHE_DIR = APP_DIR / ".clip_cache"
CACHE_DIR.mkdir(exist_ok=True)
INTERIM_DIR.mkdir(parents=True, exist_ok=True)
METADATA_CSV = DATA_DIR / "portrait_works.csv"

# -------------------- UI config --------------------
st.set_page_config(
    page_title="Embodied Aesthetic Reconstruction",
    page_icon="🖼️",
    layout="wide",
)

# -------------------- Utils --------------------
def is_icloud_placeholder(p: Path) -> bool:
    return p.suffix == ".icloud" or p.name.endswith(".icloud")

def load_image_safe(path: Path) -> Optional[Image.Image]:
    if not path.exists() or is_icloud_placeholder(path):
        return None
    try:
        with Image.open(path) as im:
            return im.convert("RGB")
    except Exception:
        return None

def pil_from_bytes(b: bytes) -> Image.Image:
    return Image.open(io.BytesIO(b)).convert("RGB")

def center_crop_long_edge(im: Image.Image, size: int = 512) -> Image.Image:
    im = ImageOps.exif_transpose(im)
    w, h = im.size
    s = min(w, h)
    left = (w - s) // 2
    top = (h - s) // 2
    im = im.crop((left, top, left + s, top + s))
    return im.resize((size, size), Image.BICUBIC)

# -------------------- Sidebar --------------------
with st.sidebar:
    st.header("⚙️ Matching Weights")
    w_clip = st.slider("CLIP weight", 0.0, 1.0, 1.00, 0.05)
    w_pose = st.slider("Pose weight", 0.0, 1.0, 0.30, 0.05)
    w_color = st.slider("Color weight", 0.0, 1.0, 0.20, 0.05)
    w_sig  = st.slider("Significance weight", 0.0, 1.0, 0.20, 0.05)

    st.markdown("---")
    st.header("🔎 Filters")
    require_public = st.checkbox("Require Public-Domain license", value=False)

    st.markdown("---")
    st.header("🧪 Debug")
    show_debug = st.checkbox("Show raw similarity score", value=False)

    st.markdown("---")
    st.header("📊 Results")
    top_k = st.slider("Top-K artworks", 1, 20, 6, 1)

    st.markdown("---")
    st.header("💻 Active device")
    st.success(f"{DEVICE}")

    st.markdown("---")
    st.header("🧍 Overlay")
    overlay_skeleton = st.checkbox("Draw skeleton on preview (if available)", value=True)

# -------------------- Metadata & dataset --------------------
def load_metadata() -> pd.DataFrame:
    """
    Expected (optional) columns:
    filename,title,artist,year,license,museum,accession,movement,
    is_masterwork,citations,exhibitions,auction_price_usd,views_per_year,
    price_estimate_usd,significance_text,interpretive_note_cn,notable_tags,source_links
    """
    if METADATA_CSV.exists():
        try:
            return pd.read_csv(METADATA_CSV)
        except Exception as e:
            st.warning(f"Could not read metadata CSV ({METADATA_CSV.name}): {e}")
    # Fallback minimal schema
    return pd.DataFrame({
        "filename": [], "title": [], "artist": [], "year": [], "license": [],
        "museum": [], "accession": [], "movement": [], "is_masterwork": [],
        "citations": [], "exhibitions": [], "auction_price_usd": [], "views_per_year": [],
        "price_estimate_usd": [], "significance_text": [], "interpretive_note_cn": [],
        "notable_tags": [], "source_links": []
    })

META = load_metadata()

def list_dataset_images() -> List[Path]:
    if not IMAGES_DIR.exists():
        return []
    files: List[Path] = []
    for ext in ("*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"):
        files += list(IMAGES_DIR.glob(ext))
    files = [p for p in files if not is_icloud_placeholder(p)]
    return sorted(files)

DATASET_FILES = list_dataset_images()

# -------------------- Significance (0..100) --------------------
TOP_MUSEUMS = ["met", "national gallery", "tate", "louvre", "uffizi", "hermitage", "ng london"]

def _logn(x, d=1.0):
    try:
        return math.log10(max(float(x), 1.0)) / d
    except Exception:
        return 0.0

def compute_significance_row(row) -> float:
    """Heuristic significance score (0..100)."""
    w = dict(master=0.35, museum=0.20, cites=0.15, exhib=0.10, auction=0.10, views=0.10)
    score = 0.0
    if str(row.get("is_masterwork", "0")).lower() in ["1", "true", "yes", "y"]:
        score += w["master"]
    museum = str(row.get("museum", "")).lower()
    if museum:
        score += w["museum"] * (1.0 if any(m in museum for m in TOP_MUSEUMS) else 0.5)
    score += w["cites"]   * _logn(row.get("citations", 0),        d=3.0)
    score += w["exhib"]   * _logn(row.get("exhibitions", 0),      d=2.0)
    score += w["auction"] * _logn(row.get("auction_price_usd",0), d=8.0)
    score += w["views"]   * _logn(row.get("views_per_year", 0),   d=6.0)
    return float(np.clip(score, 0.0, 1.0) * 100.0)

def significance_badges(row) -> List[str]:
    badges = []
    if str(row.get("is_masterwork","0")).lower() in ["1","true","yes","y"]:
        badges.append("Masterwork")
    if str(row.get("museum","")).strip():
        badges.append("Permanent Collection")
    try:
        if int(row.get("citations", 0)) >= 50:
            badges.append("Canon")
    except Exception:
        pass
    return badges[:3]

def filename_key(path_or_name: str) -> str:
    return Path(path_or_name).name

META_BY_NAME = {}
if not META.empty and "filename" in META.columns:
    for _, r in META.iterrows():
        META_BY_NAME[filename_key(str(r.get("filename","")))] = r

# -------------------- OpenCLIP --------------------
@st.cache_resource(show_spinner=False)
def load_openclip():
    try:
        import open_clip
    except Exception as e:
        st.error(f"open_clip_torch not installed: {e}")
        return None, None, None
    try:
        model_name, pretrained = "ViT-B-32", "laion2b_s34b_b79k"
        model, _, preprocess = open_clip.create_model_and_transforms(
            model_name, pretrained=pretrained, device=DEVICE
        )
        tokenizer = open_clip.get_tokenizer(model_name)
        model.eval()
        return model, preprocess, tokenizer
    except Exception as e:
        st.error(f"Failed to load OpenCLIP: {e}")
        return None, None, None

MODEL_CLIP, PRE_CLIP, TOKENIZER = load_openclip()

def tensor_from_pil_clip(im: Image.Image):
    if PRE_CLIP is None:
        return None
    t = PRE_CLIP(im).unsqueeze(0)
    if TORCH_OK:
        t = t.to(DEVICE)
    return t

@st.cache_data(show_spinner=False)
def embed_image_clip(img_bytes: bytes) -> Optional[np.ndarray]:
    if MODEL_CLIP is None:
        return None
    try:
        im = pil_from_bytes(img_bytes)
        im = center_crop_long_edge(im, size=224)
        x = tensor_from_pil_clip(im)
        with torch.no_grad():
            feat = MODEL_CLIP.encode_image(x)
            feat = feat / feat.norm(dim=-1, keepdim=True)
        return feat.cpu().numpy().astype("float32")[0]
    except Exception:
        return None

# -------------------- YOLO weights via HuggingFace (optional) --------------------
def get_yolo_pose_weights_path() -> Optional[str]:
    try:
        from huggingface_hub import hf_hub_download
    except Exception:
        return "yolov8n-pose.pt"
    try:
        local = hf_hub_download(
            repo_id="ultralytics/yolov8n-pose",
            filename="yolov8n-pose.pt",
            local_dir=str(Path.home() / ".cache" / "hf"),
            local_dir_use_symlinks=False
        )
        return local
    except Exception:
        return "yolov8n-pose.pt"

# -------------------- Feature extraction & cache --------------------
EMB_PATH   = CACHE_DIR / "embeddings.npy"
IDS_PATH   = CACHE_DIR / "ids.json"
POSE_PATH  = CACHE_DIR / "pose.npy"
COLOR_PATH = CACHE_DIR / "color.npy"
SIG_PATH   = CACHE_DIR / "significance.npy"

def _color_feature(im: Image.Image) -> np.ndarray:
    im = im.resize((256, 256), Image.BILINEAR)
    arr = np.array(im.convert("HSV"))
    h, s, v = arr[..., 0], arr[..., 1], arr[..., 2]
    hist_h, _ = np.histogram(h, bins=32, range=(0, 255), density=True)
    hist_s, _ = np.histogram(s, bins=16, range=(0, 255), density=True)
    hist_v, _ = np.histogram(v, bins=8,  range=(0, 255), density=True)
    feat = np.concatenate([hist_h, hist_s, hist_v]).astype("float32")
    return feat / (np.linalg.norm(feat) + 1e-8)

def _pose_feature_from_kpts(kpts_xyv: np.ndarray) -> np.ndarray:
    if kpts_xyv is None or len(kpts_xyv) == 0:
        return np.zeros(34, dtype="float32")
    xy = kpts_xyv[:, :2]
    min_xy = xy.min(0); max_xy = xy.max(0)
    wh = np.maximum(max_xy - min_xy, 1e-6)
    xy_n = (xy - min_xy) / wh
    feat = xy_n.reshape(-1).astype("float32")
    if feat.shape[0] < 34:
        feat = np.pad(feat, (0, 34 - feat.shape[0]))
    else:
        feat = feat[:34]
    return feat / (np.linalg.norm(feat) + 1e-8)

def _detect_pose_pil(im: Image.Image) -> Optional[np.ndarray]:
    try:
        from ultralytics import YOLO
    except Exception:
        return None
    try:
        weights = get_yolo_pose_weights_path()
        model = YOLO(weights)
        arr = np.array(im.convert("RGB"))
        res = model.predict(
            source=arr, imgsz=512, conf=0.25, verbose=False, device=yolo_device()
        )
        if not res or len(res[0].keypoints) == 0:
            return None
        kpts = res[0].keypoints.xy.cpu().numpy()
        if kpts.ndim == 4: kpts = kpts[0]
        if kpts.ndim == 3: k = kpts[0]
        else: return None
        v = np.ones((k.shape[0], 1), dtype="float32")
        kxyv = np.concatenate([k.astype("float32"), v], axis=1)
        return _pose_feature_from_kpts(kxyv)
    except Exception:
        return None

def _dataset_pose_feature(path: Path) -> Optional[np.ndarray]:
    im = load_image_safe(path)
    if im is None:
        return None
    im_small = center_crop_long_edge(im, 512)
    return _detect_pose_pil(im_small)

@st.cache_data(show_spinner=True, persist=True)
def build_dataset_embeddings(files: List[Path]):
    """Return (clip_embeds, ids, pose_feats, color_feats, signif_norm)."""
    ids = [str(p.relative_to(APP_DIR)) for p in files]

    # Use cache if aligned
    if EMB_PATH.exists() and IDS_PATH.exists() and SIG_PATH.exists():
        try:
            cached_ids = json.loads(IDS_PATH.read_text())
            if cached_ids == ids:
                clip_arr  = np.load(EMB_PATH)
                pose_arr  = np.load(POSE_PATH)  if POSE_PATH.exists()  else None
                color_arr = np.load(COLOR_PATH) if COLOR_PATH.exists() else None
                sig_arr   = np.load(SIG_PATH)
                return clip_arr, ids, pose_arr, color_arr, sig_arr
        except Exception:
            pass

    clip_feats, pose_feats, color_feats, signif_vals = [], [], [], []

    if MODEL_CLIP is None:
        st.error("OpenCLIP not ready; cannot build embeddings.")
        return np.zeros((0, 512), dtype="float32"), ids, None, None, np.zeros((0,), dtype="float32")

    progress = st.progress(0.0, text="Building dataset embeddings/features…")
    for i, p in enumerate(files):
        progress.progress((i + 1) / max(1, len(files)))
        im = load_image_safe(p)

        # CLIP
        if im is not None:
            try:
                im_clip = center_crop_long_edge(im, 224)
                x = tensor_from_pil_clip(im_clip)
                with torch.no_grad():
                    z = MODEL_CLIP.encode_image(x)
                    z = z / z.norm(dim=-1, keepdim=True)
                clip_feats.append(z.cpu().numpy().astype("float32")[0])
            except Exception:
                clip_feats.append(np.zeros(512, dtype="float32"))
        else:
            clip_feats.append(np.zeros(512, dtype="float32"))

        # Color
        if im is not None:
            try:
                color_feats.append(_color_feature(im))
            except Exception:
                color_feats.append(np.zeros(56, dtype="float32"))
        else:
            color_feats.append(np.zeros(56, dtype="float32"))

        # Pose
        pf = _dataset_pose_feature(p)
        pose_feats.append(pf if pf is not None else np.zeros(34, dtype="float32"))

        # Significance prior from metadata (0..1)
        fname = filename_key(p.name)
        row = META_BY_NAME.get(fname)
        sig = compute_significance_row(row) / 100.0 if row is not None else 0.0
        signif_vals.append(float(sig))

    clip_arr  = np.vstack(clip_feats).astype("float32") if clip_feats else np.zeros((0, 512), dtype="float32")
    pose_arr  = np.vstack(pose_feats).astype("float32") if pose_feats else None
    color_arr = np.vstack(color_feats).astype("float32") if color_feats else None
    sig_arr   = np.array(signif_vals, dtype="float32")

    # Cache
    try:
        np.save(EMB_PATH, clip_arr)
        np.save(POSE_PATH, pose_arr)
        np.save(COLOR_PATH, color_arr)
        np.save(SIG_PATH,  sig_arr)
        IDS_PATH.write_text(json.dumps(ids))
    except Exception:
        pass

    return clip_arr, ids, pose_arr, color_arr, sig_arr

# ---- Build dataset features ----
if len(DATASET_FILES) == 0:
    st.error(f"❌ No images found or folder is empty: {IMAGES_DIR}\nMake sure iCloud files are downloaded locally (no .icloud).")
else:
    st.success(f"✅ Dataset images: {len(DATASET_FILES)}")

CLIP_DS, IDS, POSE_DS, COLOR_DS, SIG_DS = build_dataset_embeddings(DATASET_FILES)  # SIG_DS in [0,1]

# ---- Ready checks ----
READY = True
reasons = []
if MODEL_CLIP is None:
    READY = False; reasons.append("OpenCLIP not loaded.")
if len(DATASET_FILES) == 0:
    READY = False; reasons.append("No images in data/images (or still .icloud).")
if CLIP_DS is None or getattr(CLIP_DS, "shape", (0,))[0] == 0:
    READY = False; reasons.append("Dataset embeddings not built yet.")
if not READY:
    st.warning("Matching is not ready: " + " ".join(reasons))
else:
    st.success("Matching is ready ✔")

# -------------------- Similarity + Resonance --------------------
def cos_sim(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    if a is None or b is None:
        return None
    if a.ndim == 1:
        a = a[None, :]
    a = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-8)
    b = b / (np.linalg.norm(b, axis=1, keepdims=True) + 1e-8)
    return (a @ b.T)

def interpret_score(score: float) -> str:
    """Map numeric similarity to a poetic/semantic label."""
    if score > 0.80: return "Strong resonance 💫"
    if score > 0.65: return "Aesthetic kinship ✨"
    if score > 0.50: return "Subtle correspondence 🌙"
    return "Distant echo 🌫️"

# -------------------- Pose overlay (preview only) --------------------
def draw_skeleton_overlay(im: Image.Image) -> Tuple[Image.Image, Optional[np.ndarray]]:
    try:
        from ultralytics import YOLO
        import cv2
    except Exception:
        return im, None

    weights = get_yolo_pose_weights_path()
    model = YOLO(weights)
    arr = np.array(im.convert("RGB"))
    res = model.predict(source=arr, imgsz=512, conf=0.25, verbose=False, device=yolo_device())
    if not res or len(res[0].keypoints) == 0:
        return im, None

    canvas = arr.copy()
    pts = res[0].keypoints.xy.cpu().numpy()
    if pts.ndim == 4: pts = pts[0]
    if pts.ndim == 3: pts = pts[0]

    PAIRS = [(5,7),(7,9),(6,8),(8,10),(11,13),(13,15),(12,14),(14,16),(5,6),(11,12),(5,11),(6,12)]
    try:
        import cv2
        for (a,b) in PAIRS:
            xa,ya = pts[a]; xb,yb = pts[b]
            cv2.line(canvas,(int(xa),int(ya)),(int(xb),int(yb)),(0,255,0),3)
        for (x,y) in pts:
            cv2.circle(canvas,(int(x),int(y)),4,(0,0,255),-1)
        im_draw = Image.fromarray(canvas)
    except Exception:
        im_draw = im

    v = np.ones((pts.shape[0],1),dtype="float32")
    kxyv = np.concatenate([pts.astype("float32"), v], axis=1)
    pf = _pose_feature_from_kpts(kxyv)
    return im_draw, pf

# -------------------- UI — Main --------------------
st.title("Embodied Aesthetic Reconstruction")
st.caption("Camera / Upload → CLIP + Pose + Color + Significance → Resonant artworks")

status_cols = st.columns(3)
with status_cols[0]:
    st.success(f"Images dir: {IMAGES_DIR.relative_to(APP_DIR)}" if IMAGES_DIR.exists() else f"Missing: {IMAGES_DIR}")
with status_cols[1]:
    st.info(f"Dataset files: {len(DATASET_FILES)}")
with status_cols[2]:
    st.info(f"Models: OpenCLIP={'✔︎' if MODEL_CLIP else '✖︎'} · Pose(ultralytics)={'✔︎'}")

st.markdown("---")

left, right = st.columns([1, 1])
query_img: Optional[Image.Image] = None
query_pose_feat: Optional[np.ndarray] = None

with left:
    st.subheader("📷 Camera (auto center-crop)")
    cam = st.camera_input("Take a photo (allow permission first)", key="camera")
    if cam is not None:
        try:
            img = pil_from_bytes(cam.getvalue())
            img = center_crop_long_edge(img, 640)
            out_path = INTERIM_DIR / "locked_frame.jpg"
            img.save(out_path, quality=92)
            st.success(f"Saved: {out_path.as_posix()}")
            if overlay_skeleton:
                img_draw, q_pose = draw_skeleton_overlay(img)
                st.image(img_draw, caption="Camera preview", use_container_width=True)
                query_img = img_draw
                query_pose_feat = q_pose
            else:
                st.image(img, caption="Camera preview", use_container_width=True)
                query_img = img
        except Exception as e:
            st.error(f"Camera decode failed: {e}")

with right:
    st.subheader("🖼️ Or upload an image")
    up = st.file_uploader("JPG/PNG", type=["jpg", "jpeg", "png"])
    if up is not None:
        try:
            img = pil_from_bytes(up.getvalue())
            img = center_crop_long_edge(img, 640)
            if overlay_skeleton:
                img_draw, q_pose = draw_skeleton_overlay(img)
                st.image(img_draw, caption="Uploaded preview", use_container_width=True)
                query_img = img_draw
                query_pose_feat = q_pose
            else:
                st.image(img, caption="Uploaded preview", use_container_width=True)
                query_img = img
        except Exception as e:
            st.error(f"Upload decode failed: {e}")

st.markdown("---")
run = st.button("🔎 Run Matching", disabled=not READY)

# -------------------- Display helpers --------------------
def display_results(idx_scores: List[Tuple[int, float]], k: int):
    if len(idx_scores) == 0:
        st.warning("No results to display.")
        return
    cols = st.columns(min(3, k))
    for i, (idx, sc) in enumerate(idx_scores[:k]):
        fn_rel = IDS[idx]
        p = (APP_DIR / fn_rel).resolve()
        im = load_image_safe(p)

        row = META_BY_NAME.get(filename_key(p.name))
        if row is not None:
            title  = str(row.get("title", "")).strip()
            artist = str(row.get("artist", "")).strip()
            year   = str(row.get("year", "")).strip()
            museum = str(row.get("museum", "")).strip()
            price  = str(row.get("price_estimate_usd", "")).strip()
            sigtxt = str(row.get("significance_text", "")).strip()
            sigcn  = str(row.get("interpretive_note_cn", "")).strip()
            links  = str(row.get("source_links", "")).strip()
            sig100 = compute_significance_row(row)
            badges = significance_badges(row)
        else:
            title=artist=year=museum=price=sigtxt=sigcn=links=""
            sig100, badges = 0.0, []

        with cols[i % len(cols)]:
            if im is not None:
                st.image(im, use_container_width=True)

            # Resonance-first display
            if show_debug:
                st.markdown(f"**Score:** {sc:.3f}")
            else:
                st.markdown(f"**{interpret_score(sc)}**")

            if title or artist or year:
                st.caption(f"{title} — {artist} ({year})")
            if museum:
                st.caption(f"🏛️ {museum}")
            if price:
                st.caption(f"💰 Estimated value: ${price}")

            st.markdown(f"**Significance:** {sig100:.0f}/100")
            if badges:
                st.caption(" · ".join(badges))
            if sigtxt:
                st.markdown(f"_{sigtxt}_")
            if sigcn:
                st.markdown(f"**Curatorial note (CN):** {sigcn}")
            if links:
                first = links.split(";")[0].strip()
                if first:
                    st.write(f"[Learn more]({first})")

# -------------------- Matching --------------------
if run:
    if query_img is None:
        st.warning("Please take a photo or upload an image first.")
    else:
        st.write("Computing embeddings…")
        t0 = time.time()

        q_clip = None
        if MODEL_CLIP is not None:
            try:
                buf = io.BytesIO()
                query_img.save(buf, format="JPEG", quality=92)
                q_clip = embed_image_clip(buf.getvalue())
            except Exception:
                q_clip = None

        try:
            q_color = _color_feature(query_img)
        except Exception:
            q_color = None

        if query_pose_feat is None and w_pose > 0:
            query_pose_feat = _detect_pose_pil(center_crop_long_edge(query_img, 512))
        w_pose_eff = w_pose if query_pose_feat is not None and POSE_DS is not None else 0.0
        if w_pose > 0 and w_pose_eff == 0.0:
            st.info("No pose feature detected or dataset pose not precomputed; treating Pose weight as 0.")

        valid_indices = np.arange(len(IDS))
        if require_public and not META.empty and {"filename", "license"}.issubset(META.columns):
            pd_mask = META["license"].astype(str).str.contains("Public Domain", case=False, na=False)
            pool_names = set(META.loc[pd_mask, "filename"].astype(str).apply(lambda s: Path(s).name))
            filt = [i for i, fn in enumerate(IDS) if Path(fn).name in pool_names]
            if len(filt) > 0:
                valid_indices = np.array(filt, dtype=int)
            else:
                st.warning("No entries meet Public-Domain filter; ignoring filter.")

        def slicer(arr):
            if arr is None or arr.shape[0] != len(IDS):
                return None
            return arr[valid_indices]

        CLIP_POOL  = slicer(CLIP_DS)
        POSE_POOL  = slicer(POSE_DS)
        COLOR_POOL = slicer(COLOR_DS)
        SIG_POOL   = SIG_DS[valid_indices] if SIG_DS is not None and SIG_DS.shape[0] == len(IDS) else None

        def cos_sim_local(q, bank):
            if q is None or bank is None:
                return None
            a = q[None, :] if q.ndim == 1 else q
            a = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-8)
            b = bank / (np.linalg.norm(bank, axis=1, keepdims=True) + 1e-8)
            return (a @ b.T)[0]

        score = np.zeros(valid_indices.size, dtype="float32")
        s = cos_sim_local(q_clip, CLIP_POOL)
        if s is not None: score += w_clip * s
        s = cos_sim_local(query_pose_feat, POSE_POOL) if w_pose_eff > 0 else None
        if s is not None: score += w_pose_eff * s
        s = cos_sim_local(q_color, COLOR_POOL)
        if s is not None: score += w_color * s
        if SIG_POOL is not None and w_sig > 0.0:
            score += w_sig * SIG_POOL

        topk_local = np.argsort(-score)[:min(top_k, score.size)]
        results = [(int(valid_indices[i]), float(score[i])) for i in topk_local]

        dt = (time.time() - t0) * 1000
        st.info(f"Search time: {dt:.1f} ms")

        display_results(results, k=top_k)

# -------------------- Tips --------------------
with st.expander("Tips"):
    st.markdown(
        """
- If you see **iCloud .icloud** placeholders, open the folder in Finder and choose **“Download Now”**.
- Adjust **CLIP / Pose / Color / Significance** weights to influence ranking.
- **Debug mode** reveals raw similarity scores; keep it off for exhibitions to maintain a poetic tone.
- The **Public-Domain** filter works only if `license` exists in your metadata CSV.
- First run builds and caches dataset features in `.clip_cache/`.
- Pose detection uses `ultralytics` (YOLOv8n-pose). Weights are pulled from Hugging Face when possible.
- *Significance* is a heuristic combining museum presence, citations, and attention. It reflects historical canons yet encourages plural aesthetics.
"""
    )


#frontend

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== 路径与常量 ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# 本地数据（用于根据 filename 找 meta）
DATA_DIR = ROOT_DIR / "data" / "local"
IMAGES_DIR = DATA_DIR / "images"

# 可能存在的 meta CSV
META_CSV_CANDIDATES = [
    DATA_DIR / "portrait_works_enhanced_english.csv",
    DATA_DIR / "portrait_works.csv",
]

# 后端 API（需要时你可以改成 HuggingFace 地址）
DEFAULT_API_URL = "http://127.0.0.1:8000/match"

APP_TITLE = "Embodied Aesthetic Reconstruction"

# YOLOv8-Pose 模型路径（放在 frontend 目录下）
YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

# WebRTC 配置（Safari 需要 STUN）
RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

# Stillness 检测（与策展版一致）
STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

# 颜色
YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

# 展示区域宽度
RIGHT_IMG_MAXW = 900


# =================== 小工具函数 ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """加载本地 CSV，按 filename 建立索引。"""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    """根据 filename 在 IMAGES_DIR 里找到图片。"""
    if not filename:
        return None
    p = Path(filename)
    if not p.is_absolute():
        p = IMAGES_DIR / p
    return p if p.exists() else None


def _load_font(size: int = 40) -> ImageFont.FreeTypeFont:
    """优先用 Courier / Courier New，找不到再退回 Arial / 默认。"""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """右上角粉色文字：姿态指标（与策展版一致）。"""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    """把关键点变成几行小字（与策展版同样结构）。"""
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """
    在画作上叠加 3 个黄色矩形标签：
    1. 价格（price_text / auction_price_usd）
    2. 年份（year）
    3. 艺术家（artist）
    字体：Courier，黑字，黄色背景，顺序固定。
    """
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    # 从画面中部略靠上开始
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        # 用 textbbox，避免 textsize 报错
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO 视频处理（完全沿用策展骨架风格） ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    与策展版一致：
    - 使用 YOLOv8-Pose 绘制蓝色框 + 绿色骨架
    - 右上角粉色文本显示姿态指标
    - 支持自动静止抓拍 & 手动抓拍
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        """把当前帧与当前指标作为一次抓拍。"""
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]  # YOLO 自带蓝框+绿骨架
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model not available)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== Streamlit 页面（策展布局） ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

# 简单 CSS：两列结构，左摄像头竖屏居中，右图像区域固定宽度
st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}
.block-container {{ padding-top: 0.6rem; padding-bottom: 0.6rem; max-width: 1700px; }}
.left-col .cam-wrap {{
  position: relative;
  height: 92vh;
  width: 100%;
  overflow: hidden;
  border-radius: 12px;
  background: #111;
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 12px !important;
}}
.right-col .art-wrap {{
  position: relative;
  height: 92vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: auto !important;
}}
</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture on demand. "
    "Left: live with pose. Right: matched artwork with tiny pink metrics."
)

# 尝试轻微自动刷新，让自动抓拍时右侧自动更新
try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None

if "last_match" not in st.session_state:
    st.session_state["last_match"] = None  # 保存最后一次 API 返回内容
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0

API_URL = DEFAULT_API_URL  # 如果你希望在 UI 里可编辑，也可以做成 text_input


with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)  # /cam-wrap
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)
    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0
    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "No frame yet, try again.", icon="✅" if ok else "⚠️")

    # 倒计时逻辑
    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Please hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "No frame yet.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)  # /left-col


with right:
    st.subheader("Matched artwork")
    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        # 从处理器读取最新抓拍
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            # 更新本地时间戳
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            # 把抓拍图发给后端 /match
            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            data = {"museum": "local", "topk": 3}

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                payload = resp.json()
                st.session_state["last_match"] = payload
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to trigger matching…")
        elif "error" in payload:
            ph.error(f"Error from backend: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned from backend.")
            else:
                # 只取 Top-1，展陈效果更干净
                top = results[0]
                filename = top.get("filename") or top.get("file")

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image file not found for: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open image: {img_path}")
                    else:
                        # 从本地 CSV 查补充 meta
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist") or top.get("artist") or "artist name",
                            "year": meta_row.get("year") or top.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)

                        # 把抓拍时的姿态指标叠到右上角（粉色文本）
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(
                            painted, metrics, size=16, margin=12
                        )

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)  # /art-wrap
    st.markdown("</div>", unsafe_allow_html=True)  # /right-col
